<a href="https://colab.research.google.com/github/Myavuzars/NLP-with-Tensorflow/blob/main/Email_classification_(Logistic_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

import tensorflow as tf
import tensorflow.keras.layers as kl
from tensorflow.keras.models import Model

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from nltk.tokenize import word_tokenize

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/Email Classification/emails.csv")
print(df.shape)
df=df.drop(df.columns[2:110], axis=1)

(5730, 110)


In [4]:
#Dealing with missing values
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
print(rows_with_NaN)

                                                   text spam
1380  Subject: from the enron india newsdesk - april...  NaN
2651  Subject: from the enron india newsdesk - april...  NaN


In [5]:
df = df.reset_index().dropna().set_index('index')

In [6]:
df.groupby('spam').describe()

text  ...     
                                                   count  ... freq
spam                                                      ...     
 its termination would not  have such a phenome...     1  ...    1
 mr suresh prabhu                                      1  ...    1
0                                                   4358  ...    2
1                                                   1368  ...    1

[4 rows x 4 columns]

In [7]:
newdf= df.loc[df['spam'].str.contains('0|1',na=False)] #OR
newdf.head(3)

,text,spam
index,,
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1


In [8]:
newdf.groupby('spam').describe()

text                                                               
     count unique                                                top freq
spam                                                                     
0     4358   4325  Subject: re : grades  pam ,  the students rese...    2
1     1368   1368  Subject: our medz  how to save on your medlcat...    1

In [14]:
#Alphabet level
numeric_data = pd.DataFrame([dict(Counter(t)) for t in newdf.text]).fillna(0)
numeric_data.head(3)


,S,u,b,j,e,c,t,:,,n,a,r,l,y,i,s,o,p,d,h,m,k,f,q,g,v,w,;,",",.,z,-,',1,0,%,x,_,$,4,...,9,3,7,2,!,(,),8,@,&,/,?,|,*,"""",+,=,>,#,[,],^,},<,~,\,`,{,,,,,,,,,,,,
0,1,38,18,1,112,34,100,9,341,58.0,80.0,62.0,50.0,28.0,77.0,60.0,91.0,15.0,33.0,30.0,29.0,9.0,22.0,3.0,22.0,12.0,17.0,3.0,4.0,10.0,1.0,4.0,1.0,1.0,2.0,1.0,1.0,104.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,20,14,1,50,16,38,1,92,44.0,43.0,36.0,29.0,11.0,42.0,26.0,38.0,16.0,13.0,15.0,13.0,10.0,5.0,0.0,11.0,1.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,12,6,1,36,6,31,1,95,21.0,30.0,17.0,11.0,10.0,25.0,14.0,36.0,10.0,13.0,11.0,12.0,3.0,9.0,0.0,4.0,6.0,8.0,0.0,2.0,3.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,2.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
newdf["spam"] = newdf["spam"].astype(str).astype(int)
labels = newdf['spam'].to_numpy()
numeric_data = numeric_data.to_numpy()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
x_train, x_test, y_train, y_test = train_test_split(numeric_data, labels)


In [17]:
#Keras
lin = kl.Input((x_train.shape[1],))
out = kl.Dense(1, activation='sigmoid')(lin)
model = Model(lin, out)
model.compile('adam','binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100)


Epoch 1/100
135/135 [==============================] - 1s 2ms/step - loss: 44.5696 - accuracy: 0.3659
Epoch 2/100
135/135 [==============================] - 0s 2ms/step - loss: 2.1748 - accuracy: 0.6763
Epoch 3/100
135/135 [==============================] - 0s 2ms/step - loss: 1.9418 - accuracy: 0.6882
Epoch 4/100
135/135 [==============================] - 0s 2ms/step - loss: 1.7521 - accuracy: 0.6956
Epoch 5/100
135/135 [==============================] - 0s 2ms/step - loss: 1.5205 - accuracy: 0.7045
Epoch 6/100
135/135 [==============================] - 0s 2ms/step - loss: 1.3408 - accuracy: 0.7147
Epoch 7/100
135/135 [==============================] - 0s 2ms/step - loss: 1.1935 - accuracy: 0.7233
Epoch 8/100
135/135 [==============================] - 0s 2ms/step - loss: 1.0827 - accuracy: 0.7313
Epoch 9/100
135/135 [==============================] - 0s 2ms/step - loss: 0.9560 - accuracy: 0.7513
Epoch 10/100
135/135 [==============================] - 0s 2ms/step - loss: 0.8942 - accur

In [18]:
print("Accuracy Score:", model.evaluate(x_test, y_test))

45/45 [==============================] - 0s 1ms/step - loss: 0.2752 - accuracy: 0.8953
Accuracy Score: [0.2751554846763611, 0.8952513933181763]


In [19]:
#Sklearn
lr = LogisticRegression()
lr.fit(numeric_data, newdf['spam'])
print(np.mean(lr.predict(x_test)==y_test))

0.8931564245810056


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
#%% Word Level
numeric_data = pd.DataFrame([dict(Counter(word_tokenize(t))) for t in newdf.text]).fillna(0)

numeric_data.head(3)


,Subject,:,naturally,irresistible,your,corporate,identity,lt,is,really,hard,to,recollect,a,company,the,market,full,of,suqgestions,and,information,isoverwhelminq,;,but,good,catchy,logo,",",stylish,statlonery,outstanding,website,will,make,task,much,easier,.,we,...,conedison,whitter,alcoa,cournerstone,dammon,rdl,3696,rgob,2302,parlour,parlourc,rout,7588,telmer,telmerc,8838,zin,szoh,waggenspack,000000000005411,yeo,naao,9683,burry,villeggiante,fredrik,eriksson,cadena,buerkle,wente,etringer,jonalan,laird,hrach,numerate,selectable,hypercube,toughest,critic,desiring
0,1,9,1.0,1.0,7.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,5.0,3.0,5.0,2.0,1.0,4.0,1.0,5.0,1.0,1.0,3.0,2.0,2.0,1.0,3.0,4.0,1.0,1.0,1.0,2.0,6.0,2.0,1.0,2.0,1.0,10.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
numeric_data = numeric_data.to_numpy()
x_train, x_test, y_train, y_test = train_test_split(numeric_data, labels)

In [23]:
#Keras
lin = kl.Input((x_train.shape[1],))
out = kl.Dense(1, activation='sigmoid')(lin)
model = Model(lin, out)
model.compile('adam','binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100)


Epoch 1/100
135/135 [==============================] - 1s 5ms/step - loss: 0.3002 - accuracy: 0.9215
Epoch 2/100
135/135 [==============================] - 1s 5ms/step - loss: 0.1374 - accuracy: 0.9881
Epoch 3/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0953 - accuracy: 0.9942
Epoch 4/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0728 - accuracy: 0.9967
Epoch 5/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0585 - accuracy: 0.9981
Epoch 6/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0486 - accuracy: 0.9986
Epoch 7/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0412 - accuracy: 0.9993
Epoch 8/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0355 - accuracy: 0.9995
Epoch 9/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0310 - accuracy: 0.9995
Epoch 10/100
135/135 [==============================] - 1s 5ms/step - loss: 0.0273 - accura

In [24]:
print("Accuracy Score:", model.evaluate(x_test, y_test))



45/45 [==============================] - 0s 3ms/step - loss: 0.0615 - accuracy: 0.9951
Accuracy Score: [0.06151460483670235, 0.9951117038726807]


In [25]:
lr = LogisticRegression()
lr.fit(numeric_data, newdf['spam'])


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [57]:
print(np.mean(lr.predict(x_test)==y_test))

0.9993016759776536
